In [4]:
import pandas as pd
import numpy as np
import os
import scipy
import string
import csv

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import warnings
from sklearn import preprocessing

import matplotlib.pyplot as plt
import plotly.graph_objects as go

warnings.filterwarnings('ignore')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Derya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Derya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Derya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
train=pd.read_csv('C:/Users/Derya/Downloads/train.csv')
train1=train.copy()
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
Q1=train1.iloc[:,[2,4]]
Q2=train1.iloc[:,[1,3]]

In [8]:
df = pd.DataFrame( np.concatenate( (Q2.values, Q1.values), axis=0 ) )
df.columns = ['id','question' ]

In [9]:
df

,id,question
0,1,What is the step by step guide to invest in sh...
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,5,How can I increase the speed of my internet co...
3,7,Why am I mentally very lonely? How can I solve...
4,9,"Which one dissolve in water quikly sugar, salt..."
...,...,...
808575,379845,How many keywords are there in PERL Programmin...
808576,155606,Is it true that there is life after death?
808577,537929,What's this coin?
808578,537931,I am having little hairfall problem but I want...


In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [14]:
tok_quora=[word_tokenize(str(wrd)) for wrd in df.question]

In [ ]:
Quora_training_data=[TaggedDocument(d, [i]) for i, d in enumerate(tok_quora)]

In [ ]:
doc_model = Doc2Vec(Quora_training_data, vector_size = 100, window = 5, min_count = 3, epochs = 25)

In [ ]:
def fetch_embeddings(model,tokens):
    tokens = [x for x in word_tokenize(tokens) if x in list(doc_model.wv.vocab)]
 
    if len(tokens)>=1:
        return doc_model.infer_vector(tokens)
    else:
        return np.array([0]*100

In [ ]:
doc_embeddings=[]
for w in df.question:
    doc_embeddings.append(list(fetch_embeddings(doc_model, w)))
#conveting it into array
doc_embeddings=np.asarray(doc_embeddings)
#shape
Shape=(10000,100)

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('paraphrase-MiniLM-L12-v2')
x=[i for i in df.question]
#lets get embeddings for each question
sentence_embeddings_BERT= sbert_model.encode(x)
#lets see the shape
sentence_embeddings_BERT.shape

In [ ]:
!pip install pytorch_pretrained_bert

In [ ]:
import torch from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel
tok_gpt= OpenAIGPTTokenizer.from_pretrained('openai-gpt')

model_gpt= OpenAIGPTModel.from_pretrained('openai-gpt')
model_gpt.eval()

def Fetch_gpt_vectors(question):
 #tokenize words
    words = word_tokenize(question)
    emb = np.zeros((1,768))
 #get vectore for each word
    for word in words:
        w= tok_gpt.tokenize(word)
    indexed_words = tok_gpt.convert_tokens_to_ids(w)
    tns_word = torch.tensor([indexed_words])
    with torch.no_grad():
        try:
 #get mean vector
           emb += np.array(torch.mean(model_gpt(tns_word),1))
        except Exception as e:
           continue
    emb /= len(words)
    return emb

In [ ]:
gpt_emb = np.zeros((1000, 768))
for v in range(1000):
    txt = df.loc[v,'question']
    gpt_emb[v] = Fetch_gpt_vectors(txt)
gpt_emb

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

In [ ]:
def cosine_similarity(vec1,vec2):
 
    return dot(vec1, vec2)/(norm(vec1)*norm(vec2))

In [ ]:
def top_n_questions(user,embeddings,df):
    x=cosine_similarity(user,embeddings).tolist()[0]
    temp_list=list(x)
 #sorting
    sort_res = sorted(range(len(x)), key = lambda sub: x[sub])[:]
    sim_score=[temp_list[i] for i in reversed(sort_res)]
 #print
    print(sort_res[0:5])
 #index fetching
    L=[]
    for i in reversed(sort_res):
        L.append(i)
 #get the index from dataframe
    return df.iloc[L[0:5], [0,1]]

In [ ]:
def get_input_vector(query,model):
    print(query)
 #Doc2vec model
    if model=='Doc2Vec':
        k=fetch_embeddings(doc_model,query)
        k=k.reshape(1, -1)
    elif model=='BERT':
        k=sbert_model.encode(str(query))
        k=k.reshape(1, -1)
 # gpt model
    elif model=='GPT':
        k=Fetch_gpt_vectors(query)
    return k

In [ ]:
top_n_questions(get_input_vector('How is Narendra Modi as a person?','Doc2Vec'),doc_embeddings,df)

In [ ]:
top_n_questions(get_input_vector('How is Narendra Modi as a person?','GPT'),gpt_emb,df)

In [ ]:
top_n_questions(get_input_vector('How is Narendra Modi as a person?','BERT'),sentence_embeddings_BERT,df)

In [ ]:
import re
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Dropout, LSTM

In [ ]:
train = pd.read_csv('C:/Users/Derya/Downloads/train.csv')
data = train.copy()


In [ ]:
def txt_process(input_text):
 # Removing punctuation from input text
    input_text = ''.join([x for x in input_text if x not in punctuation])
 # Cleaning the text
    input_text = re.sub(r"[^A-Za-z0-9]", " ", input_text)
    input_text = re.sub(r"\'s", " ", input_text)
 # remove stop words
    input_text = input_text.split()
    input_text = [x for x in input_text if not x in stop_words]
    input_text = " ".join(input_text)
 # Return a list of words
    return(input_text)

In [ ]:
data['question1_cleaned'] = data.apply(lambda x: txt_process(x['question1']), axis = 1)
data['question2_cleaned'] = data.apply(lambda x: txt_process(x['question2']), axis = 1)

In [ ]:
question_text = np.hstack([data.question1_cleaned, data.question2_cleaned])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(question_text)

In [ ]:
data['tokenizer_1'] = tokenizer.texts_to_sequences(data.question1_cleaned)
data['tokenizer_2'] = tokenizer.texts_to_sequences(data.question2_cleaned)
data.head(5)

In [ ]:
data['tokenizer'] = data['tokenizer_1'] + data['tokenizer_2']
m_len = 500

max_token = np.max(data.tokenizer.max())

In [ ]:
y = data[['is_duplicate']]
X = data[['tokenizer']]
#padding X with a maximum length
X = sequence.pad_sequences(X.tokenizer, maxlen = m_len)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=10)

In [ ]:
quora_model = Sequential()
#adding embeedding layer
quora_model.add(Embedding(70000, 64))
#adding drop out layer
quora_model.add(Dropout(0.15))
#LSTM layer
quora_model.add(LSTM(16))
#adding sigmoid layer
quora_model.add(Dense(1, activation = 'sigmoid'))
#defining loss and optimizer
quora_model.compile(loss='binary_crossentropy', optimizer='SGD',metrics=['accuracy'])
quora_model.summary()

In [ ]:
quora_model.fit(X_train, y_train, epochs = 2, batch_size=64,validation_data=(X_test,y_test))

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
tr_prediction=quora_model.predict(X_train)

In [ ]:
tr_prediction[tr_prediction>0.5]=1
tr_prediction[tr_prediction<0.5]=0
tr_prediction

In [ ]:
tr_true=y_train.values
#accuracy
Accuracy=sklearn.metrics.accuracy_score(np.array(tr_true), np.array(tr_prediction))
print(Accuracy)

In [ ]:
print(classification_report(tr_true, tr_prediction, target_names=['Not similar','similar']))

In [ ]:
test_prediction=quora_model.predict(X_test)

In [ ]:
test_prediction[test_prediction>0.5]=1
test_prediction[test_prediction<0.5]=0
test_prediction
#true values for test
test_true=y_test.values
# accuracy on test data
Accuracy=sklearn.metrics.accuracy_score(np.array(test_true), np.array(test_prediction))
print('Accuracy is %f'%(Accuracy*100)+' %')

print(classification_report(test_true, test_prediction, target_names=['Not similar','similar']))